In [1]:
from brevitas.core.scaling import ConstScaling
from brevitas.core.quant.int_base import IntQuant
import torch
import torch.nn as nn
from torch.nn.functional import linear
import torch.optim as optim
import torch.nn.functional as F
import argparse
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [2]:
int_quant = IntQuant(narrow_range=True, signed=False)
print(torch.tensor(4.))
scale, zero_point, bit_width = torch.tensor(0.01), torch.tensor(0.), torch.tensor(2.)
inp = torch.Tensor([0.042, -0.053, 0.31, -0.44])
out = int_quant(scale, zero_point, bit_width, inp)
print(out)

tensor(4.)
tensor([0.0200, 0.0000, 0.0200, 0.0000])


Función para backward hook


In [3]:
def printgradnorm(self, grad_input, grad_output):
    print('Inside ' + self.__class__.__name__ + ' backward')
    print('Inside class:' + self.__class__.__name__)
    print('')
    """
    print('grad_input: ', type(grad_input))
    print('grad_input[0]: ', type(grad_input[0]))
    print('grad_output: ', type(grad_output))
    print('grad_output[0]: ', type(grad_output[0]))
    print('')
    print('grad_input size:', grad_input[0].size())
    print('grad_output size:', grad_output[0].size())
    print('grad_input norm:', grad_input[0].norm())
    print(grad_input[0])"""

    #print(type(grad_output[0]))
    #grad_input[0][:] = 5#torch.round(input=grad_input[0],decimals=5)
    if type(grad_input[0]) == torch.Tensor:
        #print("Antes: ",grad_output[0])
        grad_output[0][:] = 5
        #print("Despues:",grad_output[0])


Definición de argumentos y datos a usar

In [4]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        #print(model.linear_relu_stack[0].weight.grad)
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if args.dry_run:
                break


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
# Training settings
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
            help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
            help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=6, metavar='N',
            help='number of epochs to train (default: 14)')
parser.add_argument('--lr', type=float, default=1.0, metavar='LR',
            help='learning rate (default: 1.0)')
parser.add_argument('--gamma', type=float, default=0.7, metavar='M',
            help='Learning rate step gamma (default: 0.7)')
parser.add_argument('--no-cuda', action='store_true', default=False,
            help='disables CUDA training')
parser.add_argument('--dry-run', action='store_true', default=False,
            help='quickly check a single pass')
parser.add_argument('--seed', type=int, default=1, metavar='S',
            help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
            help='how many batches to wait before logging training status')
parser.add_argument('--save-model', action='store_true', default=False,
            help='For Saving the current Model')
args = parser.parse_args("")
use_cuda = not args.no_cuda and torch.cuda.is_available()

print(args)

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': args.batch_size}
test_kwargs = {'batch_size': args.test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                'pin_memory': True,
                'shuffle': True}
train_kwargs.update(cuda_kwargs)
test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    #media y desviación típica de la base de datos MNIST
    transforms.Normalize((0.1307,), (0.3081,))
    ])

dataset1 = datasets.MNIST('./data', train=True, download=True,
            transform=transform)
dataset2 = datasets.MNIST('./data', train=False,
            transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

Namespace(batch_size=64, dry_run=False, epochs=6, gamma=0.7, log_interval=10, lr=1.0, no_cuda=False, save_model=False, seed=1, test_batch_size=1000)


Creación de la linear custom layer

In [19]:
"""class LinearC(nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = True,device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(LinearC,self).__init__()
        self.in_features = in_features
        self.size_out = out_features
        self.weight = nn.Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        nn.init.xavier_normal_(self.weight,1)

    def forward(self,x):
        output = linear(x,self.weight,bias=None)
        return torch.round(input=output,decimals=3)"""

class LinearC(nn.Linear):
    def __init__(self, in_features: int, out_features: int, bias: bool = True,device=None, dtype=None) -> None:
        super().__init__(in_features,out_features,bias=bias,device=device,dtype=dtype)
        

    def forward(self,x):
        output = super().forward(x)
        #print(output)
        return output#torch.round(input=output,decimals=5)    


In [12]:
class CustomNet(nn.Module):
    def __init__(self):
        super(CustomNet, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,4),
            nn.ReLU(),
            nn.Linear(4,10)
        )

    def forward(self,x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        return F.log_softmax(x,dim=1)
        
    

In [8]:
#net.linear_relu_stack[0].register_full_backward_hook(printgradnorm)
#net.linear_relu_stack[2].register_full_backward_hook(printgradnorm)

In [13]:
model = CustomNet().to(device)



In [14]:
optimizer = optim.Adadelta(model.parameters(), lr=0.001)


In [ ]:
train(args, model, device, train_loader, optimizer, 1)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.216807
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.150828
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.285417
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.240645
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.183556
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.190336
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.229715
Train Epoch: 1 [4480/60000 (7%)]	Loss: 2.120216
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.161169
Train Epoch: 1 [5760/60000 (10%)]	Loss: 2.252653
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.203687
Train Epoch: 1 [7040/60000 (12%)]	Loss: 2.211934
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.219008
Train Epoch: 1 [8320/60000 (14%)]	Loss: 2.293447
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.160741
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2.226797
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.249928
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.267389
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.210381
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.224570
Train Epoch: 1 [12800/60000 (

KeyboardInterrupt: 

In [ ]:
optimizer.zero_grad()
output = net(tensor)
loss = F.nll_loss(output,target)
loss.backward()
print(net.linear_relu_stack[0].weight.grad)
print(net.linear_relu_stack[2].weight.grad)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [ ]:
def modify(tensor):
    tensor[1,1] = 0

prueba = torch.ones(2,2)
print(prueba)
modify(prueba)
prueba

tensor([[1., 1.],
        [1., 1.]])


tensor([[1., 1.],
        [1., 0.]])